In [2]:
pip install google-cloud-bigquery

     |████████████████████████████████| 203 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 4.0 MB 74.1 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 79.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 78.2 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 72.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 64.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 10.2 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.15.8
    Uninstalling protobuf-3.15.8:
      Successfully uninstalled protobuf-3.15.8
Note: you may need to restart the kernel to use updated packages.


In [16]:
from google.cloud import bigquery

def upload_data_to_big_query(table_name):
# upload to BigQuery
    client = bigquery.Client(project="mythical-patrol-219308")  

    table_ref = client.dataset("second_dataset").table(table_name)  

    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.allow_quoted_newlines = True
    job_config.skip_leading_rows = 1 
    job_config.autodetect = True

    with open("/home/jovyan/data/Assignment-02/{}.csv".format(table_name), "rb") as source_file:
        job = client.load_table_from_file(
            source_file, table_ref, job_config=job_config
        )
    print(table_name)
    job.result()

list_tables = ['Donations', 'Donors','Projects', 'Resources', 'Schools', 'Teachers']
for table in list_tables:
    upload_data_to_big_query(table)

Donations
Donors
Projects
Resources
Schools
Teachers


In [17]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Ass_02")
sparkConf.set("spark.driver.memory", "4g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

def read_df(table_name):
    df = spark.read \
      .format("bigquery") \
      .load("mythical-patrol-219308.second_dataset.{}".format(table_name))  
    return df

dic_dfs = {}
for table in list_tables:
    print(table)
    dic_dfs[table] = read_df(table)
    dic_dfs[table].printSchema()

Donations
root
 |-- Project_ID: string (nullable = true)
 |-- Donation_ID: string (nullable = true)
 |-- Donor_ID: string (nullable = true)
 |-- Donation_Included_Optional_Donation: boolean (nullable = true)
 |-- Donation_Amount: double (nullable = true)
 |-- Donor_Cart_Sequence: long (nullable = true)
 |-- Donation_Received_Date: timestamp (nullable = true)

Donors
root
 |-- Donor_ID: string (nullable = true)
 |-- Donor_City: string (nullable = true)
 |-- Donor_State: string (nullable = true)
 |-- Donor_Is_Teacher: boolean (nullable = true)
 |-- Donor_Zip: string (nullable = true)

Projects
root
 |-- Project_ID: string (nullable = true)
 |-- School_ID: string (nullable = true)
 |-- Teacher_ID: string (nullable = true)
 |-- Teacher_Project_Posted_Sequence: long (nullable = true)
 |-- Project_Type: string (nullable = true)
 |-- Project_Title: string (nullable = true)
 |-- Project_Essay: string (nullable = true)
 |-- Project_Short_Description: string (nullable = true)
 |-- Project_Need_S

### Finding Schools with the most and the least total amount donation for each city

In [18]:
from pyspark.sql.functions import col, sum
joinExpression = ["Project_ID"] 
schools = dic_dfs['Schools']
projects = dic_dfs['Projects']
donation = dic_dfs['Donations']
school_amount_donation = projects.join(donation, joinExpression,"left").groupby('School_ID').agg(sum(col('Donation_amount')).alias('total_donation_amount'))

In [19]:
joinExpression = ["School_ID"] 
school_city = school_amount_donation.join(schools, joinExpression,"left")

In [20]:
from pyspark.sql.functions import *
from pyspark.sql import Row, Window

windowasc = Window.partitionBy(col("School_City")).orderBy(col("total_donation_amount").asc_nulls_last())
windowdesc = Window.partitionBy(col("School_City")).orderBy(col("total_donation_amount").desc_nulls_last())

school_windowed = school_city.withColumn("rank_amount_desc", dense_rank().over(windowdesc)).withColumn("rank_amount_asc", dense_rank().over(windowasc))
most_least_donation_school_city = school_windowed.where((col("rank_amount_desc") == 1) | (col("rank_amount_asc") == 1) ).withColumn("school_category", when(col("rank_amount_desc") == 1, "Most_amount_Donation")
                                                          .otherwise("Least_amount_Donation")).select("School_City", "School_Name","total_donation_amount","school_category")
most_least_donation_school_city.show(10)

+------------+--------------------+---------------------+--------------------+
| School_City|         School_Name|total_donation_amount|     school_category|
+------------+--------------------+---------------------+--------------------+
|      Agawam|Roberta G Doering...|                 20.0|Least_amount_Dona...|
|      Agawam|Benjamin J Phelps...|              5660.08|Most_amount_Donation|
|      Aitkin|Rippleside Elemen...|    6793.610000000001|Most_amount_Donation|
|     Antwerp|Antwerp Local School|   1413.0499999999997|Most_amount_Donation|
|Belle Plaine|Belle Plaine High...|                 50.0|Least_amount_Dona...|
|Belle Plaine|Chatfield Element...|               803.52|Most_amount_Donation|
|   Birchwood|    Birchwood School|               412.86|Most_amount_Donation|
|    Bluffton|Okatie Elementary...|                 50.0|Least_amount_Dona...|
|    Bluffton|Red Cedar Element...|              3206.85|Most_amount_Donation|
|  Blythewood|Westwood High School|                8

In [21]:
# save BigQuery
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

bucket = "temp_assignment_2"
spark.conf.set('temporaryGcsBucket', bucket)
# Saving the data to BigQuery
most_least_donation_school_city.write.format('bigquery').option('table', 'mythical-patrol-219308.second_dataset.result_school').save()

In [90]:
# saving to Bucket
most_least_donation_school_city.write.format("csv").options(header='True').save("gs://out_assignment_2/result_school.csv")


In [ ]:
spark.stop()